In [27]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

URL = "https://coronavirus.ohio.gov/wps/portal/gov/covid-19/"
r = requests.get(URL) 

r.content

b'<!DOCTYPE html>\n<html class="no-js" lang="en">\n<head>\n<meta charset="utf-8">\n<!-- rel=dynamic-content indicates an element that is replaced with the contents produced by the specified href. \n\t dyn-cs:* URIs are resolved using the WP DynamicContentSpotMappings resource environment provider. These values can\n\t also be set using theme metadata if a theme is specified in the URI (e.g. @tl:oid:theme_unique_name). ODX v1.02 -->\n<link rel="stylesheet" href="/wps/contenthandler/gov/!ut/p/digest!nz2aAYDTtmlgy3mqEZ50rw/sp/mashup:ra:collection?soffset=0&amp;eoffset=5&amp;themeID=ZJ_O9LGHI40NG82E0QG0O0KF412S1&amp;locale=en&amp;mime-type=text%2Fcss&amp;lm=1569729454000&amp;entry=wp_toolbar_common__0.0%3Ahead_css&amp;entry=wp_portlet_css__0.0%3Ahead_css&amp;entry=wp_simple_contextmenu_css__0.0%3Ahead_css&amp;entry=wp_status_bar__0.0%3Ahead_css" type="text/css"/><script type="text/javascript">(function(){if (typeof(wpModules) == \'undefined\') wpModules = {}; if (typeof(wpModules.state) ==

In [103]:
soup = BeautifulSoup(r.content, 'html.parser')
infectedDiv = soup.findAll("div", {"class": "odh-ads__super-script-item"})[0]
deathDiv = soup.findAll("div", {"class": "odh-ads__super-script-item"})[1]
infectedCounties = infectedDiv.text.split(":")[1].strip().split(',')
deathCounties = deathDiv.text.split("**")[1].strip().split(',')

In [119]:
ohioCounties = pd.read_csv('ohio_county_list.csv')
ohioCounties.columns = ['County', 'Coordinates', 'Population']
ohioCounties['County'] = ohioCounties['County'].apply(lambda x: x.split()[0].strip())

confirmedCases = dict()
for county in infectedCounties:
    info = county.split()
    info[0] = info[0].strip()
    info[1] = int(re.match("\((\d+)\)", info[1])[1])
    confirmedCases[info[0]] = info[1]

confirmedDF = pd.DataFrame.from_dict(confirmedCases, orient='index').reset_index()
confirmedDF.columns = ["County", "Confirmed"]

deaths = dict()
for county in deathCounties:
    info = county.split()
    info[0] = info[0].strip()
    info[1] = int(re.match("\((\d+)\)", info[1])[1])
    deaths[info[0]] = info[1]

deathDF = pd.DataFrame.from_dict(deaths, orient='index').reset_index()
deathDF.columns = ["County", "Deaths"]

ohioCounties = ohioCounties.join(confirmedDF.set_index('County'), on='County')
ohioCounties = ohioCounties.join(deathDF.set_index('County'), on='County')
ohioCounties['Latitude'] = ohioCounties['Coordinates'].apply(lambda x: (re.match('Point\(([^()]+)\)', x)[1].split())[1])
ohioCounties['Longitude'] = ohioCounties['Coordinates'].apply(lambda x: (re.match('Point\(([^()]+)\)', x)[1].split())[0])
ohioCounties = ohioCounties.drop(['Coordinates'], axis=1)
ohioCounties.fillna(0.0)

,County,Population,Confirmed,Deaths,Latitude,Longitude
0,Adams,28105,0.0,0.0,38.84,-83.48
1,Erie,76048,1.0,1.0,41.51,-82.61
2,Brown,44264,0.0,0.0,38.93,-83.87
3,Lucas,436393,9.0,1.0,41.68,-83.5
4,Henry,28092,0.0,0.0,41.33,-84.07
5,Franklin,1212263,44.0,2.0,39.97,-83.01
6,Clinton,41945,1.0,0.0,39.41,-83.81
7,Jefferson,67964,0.0,0.0,40.38,-80.76
8,Hamilton,804520,26.0,0.0,39.2,-84.54
9,Cuyahoga,1263154,149.0,1.0,41.54,-81.66
